# Setup

Before start, please make sure you have all the dependencies installed.

In [ ]:
!pip install -U imjoy zarr scikit-image fsspec aiohttp

If you are running inside **Google Colab**, please also run the following cell.

(Skip for Jupyter notebooks)

In [ ]:
!git clone https://github.com/hms-dbmi/vizarr
import os
os.chdir('vizarr/example')

## Zoomable Mandelbrot Set

This notebook a contains `vizarr` example visualizing a generic multiscale zarr. The first cell contains code to create the underlying generatvie zarr store. It dynamically creates "chunks" at different zoom levels and associated array metadata.

In [ ]:
import numpy as np
from numba import njit
from zarr.util import json_dumps

ZARR_FORMAT = 2
ZARR_META_KEY = ".zattrs"
ZARR_ARRAY_META_KEY = ".zarray"
ZARR_GROUP_META_KEY = ".zgroup"
ZARR_GROUP_META = {"zarr_format": ZARR_FORMAT}


def create_array_meta(shape, chunks, compressor, dtype):
    return {
        "chunks": chunks,
        "compressor": compressor.get_config() if compressor else None,
        "dtype": dtype.str,
        "fill_value": 0.0,
        "filters": None,
        "order": "C",
        "shape": shape,
        "zarr_format": ZARR_FORMAT,
    }


def create_root_attrs(levels):
    datasets = [{"path": str(i)} for i in range(levels)]
    return {"multiscales": [{"datasets": datasets, "version": "0.1"}]}


@njit
def mandelbrot(out, from_x, from_y, to_x, to_y, grid_size, maxiter):
    step_x = (to_x - from_x) / grid_size
    step_y = (to_y - from_y) / grid_size
    creal = from_x
    cimag = from_y
    for i in range(grid_size):
        cimag = from_y
        for j in range(grid_size):
            nreal = real = imag = n = 0
            for _ in range(maxiter):
                nreal = real * real - imag * imag + creal
                imag = 2 * real * imag + cimag
                real = nreal
                if real * real + imag * imag > 4.0:
                    break
                n += 1
            out[j * grid_size + i] = n
            cimag += step_y
        creal += step_x
    return out


@njit
def tile_bounds(level, x, y, max_level, min_coord=-2.5, max_coord=2.5):
    max_width = max_coord - min_coord
    tile_width = max_width / 2 ** (max_level - level)
    from_x = min_coord + x * tile_width
    to_x = min_coord + (x + 1) * tile_width

    from_y = min_coord + y * tile_width
    to_y = min_coord + (y + 1) * tile_width

    return from_x, from_y, to_x, to_y


class MandlebrotStore:
    def __init__(self, levels, tilesize, maxiter=255, compressor=None):
        self.levels = levels
        self.tilesize = tilesize
        self.compressor = compressor
        self.dtype = np.dtype(np.uint8 if maxiter < 256 else np.uint16)
        self.maxiter = maxiter
        self._store = self._init_store()

    def __getitem__(self, key):
        if key in self._store:
            return self._store[key]

        try:
            # Try parsing pyramidal coords
            level, chunk_key = key.split("/")
            level = int(level)
            y, x = map(int, chunk_key.split("."))
        except:
            raise KeyError

        from_x, from_y, to_x, to_y = tile_bounds(level, x, y, self.levels)
        out = np.zeros(self.tilesize * self.tilesize, dtype=self.dtype)
        tile = mandelbrot(out, from_x, from_y, to_x, to_y, self.tilesize, self.maxiter)

        dbytes = tile.tobytes()

        if self.compressor:
            return self.compressor.encode(dbytes)

        return dbytes

    def _init_store(self):
        d = dict()
        base_width = self.tilesize * 2 ** self.levels
        d[ZARR_GROUP_META_KEY] = json_dumps(ZARR_GROUP_META)
        d[ZARR_META_KEY] = json_dumps(create_root_attrs(self.levels))

        for level in range(self.levels):
            width = int(base_width / 2 ** level)
            array_meta = create_array_meta(
                shape=(width, width),
                chunks=(self.tilesize, self.tilesize),
                compressor=self.compressor,
                dtype=self.dtype,
            )
            d[f"{level}/{ZARR_ARRAY_META_KEY}"] = json_dumps(array_meta)

        return d

    def keys(self):
        return self._store.keys()

    def __iter__(self):
        return iter(self._store)

### Running vizarr

Simply initalize the multiscale store implemented above, and open as a `zarr.Group` for vizarr. 

In [ ]:
from imjoy_plugin import run_vizarr
from numcodecs import Blosc
import zarr

# Initialize the store
store = MandlebrotStore(levels=40, tilesize=1024, compressor=Blosc())
# Wrap in a cache so that tiles don't need to be computed as often
store = zarr.LRUStoreCache(store, max_size=2 ** 38)

# This store implements the 'multiscales' zarr specfiication which is recognized by vizarr
z_grp = zarr.open(store, mode="r")

img = { "source": z_grp, "name": "mandelbrot" }

run_vizarr(img)